<a href="https://colab.research.google.com/github/crew-guy/Machine-Learning/blob/main/tensorflow/tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook, we'll be covering


1. Intro to tensors
2. Getting info from tensors
3. Manipulating tensors
4. Tensors & numpy
5. Using @tf.function (speed up python functions)
6. Using GPUs with TF (or TPUs)

# Introduction to tensors

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
# Create tensor with tf.constant()
# Scalar : tensor of 0th dimension
scalar = tf.constant(7)
scalar  

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
scalar.ndim


0

In [4]:
# Vector : tensor with 1-dimension
vector = tf.constant([10, 10])
vector
vector.ndim

1

In [5]:
# Matrix : tensor with 2-dimensions
matrix = tf.constant([
                      [1,2],
                      [3,4]
])
matrix
matrix.ndim

2

In [7]:
tensor = tf.constant(
    [
     [
      [1,2,3],
      [4,5,6]
     ],
     [
      [7,8,9],
      [10,11,12]
     ]
    ]
)
tensor
tensor.ndim

3

In [10]:
another_matrix = tf.constant(
    [[1.,2.,3.],[4.,5.,6.]],
    dtype=tf.float16
)
another_matrix

<tf.Tensor: shape=(2, 3), dtype=float16, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float16)>

## Creating tensors with tf variable

In [11]:
# Create the same tensor with tf.Variable()
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor[0].assign(7)
changeable_tensor

# Can't change an unchangeable tensor with assignment 

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

<hr/>

🔑 **Note** : Rarely in practice, we decide, whether to use `tf.constant()` or `tf.Variable`, as Tensorflow does this for use. However, if in doubt, use `tf.constant` and change it later

<hr/>

## Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers

In [13]:
# Create 2 random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) #set seed for reproducability
random_1 = random_1.normal(shape=(3,2))

random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal ?
random_1, random_2, random_1 == random_2


(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

<hr/>

### Shuffle the order of tensors

Shuffle data so the inherent order of data doesn't affect learning


In [15]:
not_shuffled  = tf.constant([[10,7,3], [2,4,5]])
not_shuffled 

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[10,  7,  3],
       [ 2,  4,  5]], dtype=int32)>

In [22]:
# tf.random.set_seed(42) #global level random seed
shuffled = tf.random.shuffle(not_shuffled, seed=42) # operation level random seed
shuffled

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[10,  7,  3],
       [ 2,  4,  5]], dtype=int32)>

🛠 **Read** - Global & operation level seed - [Link](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

If we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed

> Rule 4 : If both global & the operation seed are set : Both seeds are use in conjunction to determine the random sequence

Because a NN initialized itself with random patterns, you could get different results, everytime you run this experiment. So, to make reproducible experiments, you probably want to initialize your data in a similar fashion each time

### Other ways to make tensors

In [23]:
# Create a tensor with all ones
tf.ones(shape=(10,7))

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [24]:
tf.zeros(shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors
The main difference between a NumPy array & Tensorflow tensors is that tensors can be run on a GPU (much faster for numeric computing)

In [29]:
import numpy as np
numpy_A = np.arange(1,25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [31]:
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(2,3,4))
A, B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

💡 **Note** : We usually use capital letters for variables storing matrices & lowercase letters for variables storing vectors

### Getting information from tensors

